In [2]:

#Import Necessary Libraries Part 1
# import h5py
import subprocess
#import dask.dataframe as daf
# from numba import njit, vectorize, cuda
import time
from scipy.fft import fft, ifft
import tables
import itertools
import scipy.sparse as sp

In [6]:
#Import Necessary Libaries Part 2
import pandas as pd
import datetime
import numpy as np
import scipy as scipy
# import matplotlib.pyplot as plt
# from matplotlib import image as mpimg
# import seaborn as sns
import scipy.io as scipio
from PIL import Image
import glob
import os
import ntpath as ntp
import csv
import statistics
import pymongo
import json
import multiprocessing as mp
from scipy.io import loadmat
import time
from multiprocessing import Pool

In [7]:
####IMPORTANT
#REFER to README.txt for setup protocols
pn = os.cpu_count()# to get the cpu core count

print ("Pipeline will run with", pn, "process nodes (number of system cores)")

def switch_slash(path):
    new = path.replace(os.sep, '/')
    return (new)

temp_store = "C:/Users/Peter/Desktop/DATA_TEST"

code = "C:/Users/Peter/Dropbox (Grant Brown's Lab)/Peter Bartlett Data/Code"

#Set the location of the experimental flow file/condtion information
locationC = "F:/Microfluidics/DATA_TEST/20210112_BaselMicrofluidics_GeneMap.xlsx" 

#Fill in the fluorescent information
Fluorescent = {'GFP': 'PROT', 'mKa': 'BHY175_Myo1', 'mKO': 'BHY131_Myo1'} 

#Set the image path to the location of the raw image files (Largest directory)
# image_path = "C:/Users/Peter/Desktop/Subset_Images/Raw" ########################################################### SUBSET
# image_path = "D:/Microfluidics/Missing_Analyze"
image_path = str(input("Image path?"))
image_path = switch_slash(image_path)

# "D:/Microfluidics/Analyze" # This would normally be a folder containing all date folders with the images aquired at those days {[All]/[2018_x]/[images...]}

#Set the Results directory. This is where the CellX will output to and where the Master hdf5 file will be stores
# microfluidics_results = "C:/Users/Peter/Desktop/Subset_Images/DEC15RESULTS" ########################################################### SUBSET
microfluidics_results = str(input("microfluidics_results"))
microfluidics_results = switch_slash(microfluidics_results)
# microfluidics_results = "D:/Microfluidics/RES_N_ULTS"

# "D:/Microfluidics/RESULTS" #This is a folder containing all image masks and other output from CellX. It should be of shape ALL/Date/[GFP],[RFP].../  AND the HDF5 file in my case

#Where are the CellX parameters stored
AllPoint = "C:/Users/Peter/OneDrive - University of Toronto/Documents/All_Cellx_Point"

#Set the location of the CellX program and the MCR version folder. On Linux, the CellX will be an .sh file
# CellXc = "C:/Users/Peter/CellX/Windows_7_g1.12_c1.12/CellX.exe"
#MCRc = "C:/Program Files (x86)/MATLAB/MATLAB Compiler Runtime/v716"

Pipeline will run with 8 process nodes (number of system cores)


In [4]:
image_path

'D:/Microfluidics/Missing_Analyze'

In [5]:
Experimental_info = {
    "timepoint_space": 7.5,
    "cell_tracking": True,
    "fluorescent": True,
    "data_subset": False
    }

In [2]:
#Get the ask fucntion to work better. Right now the ask function does not properly deliminate between requirments. Manually enter requirement in the pandas selection parameters later on for now. 
#Get the ask fucntion to work better. Right now the ask function does not properly deliminate between requirments. Manually enter requirement in the pandas selection parameters later on for now. 
if Experimental_info["data_subset"] == True:
    subdat = str(input("Enter position barcodes to be analyzed (comma seperated and in the form dMMDDpPosition)"))

    instances = subdat.split(", ")
    print(f"Analysis will be performed for positions which correspond to the following pos_barcodes: {instances}")
else:
    pass

try: subdat
except NameError: pass
else: del subdat

#     subdat = str(input("Enter position barcodes to be analyzed (comma seperated and in the form d*MMDD*r*RunNumber*p*Position*"))
#     instances = subdat.split(", ")
#     return(instances)
#     print(f"Analysis will be performed for positions which correspond to the following pos_barcodes: {instances}")


# if Experimental_info["data_subset"] == True:
#     instances = subdat_q()
#  else:
#      pass

# Permission = str(input("Is this correct? (Y or N)"))
# if Permission == "Y":
#     pass
# else:
#     isinstances = subdat_q()


# try: subdat
# except NameError: pass
# else: del subdat

NameError: name 'Experimental_info' is not defined

In [7]:
# def f_pos_to_int(pos):
#     integer = int(pos)
#     return(integer)

In [8]:
image_path

'D:/Microfluidics/Missing_Analyze'

In [10]:
##Layer : Creating a file matrix with component information and matching to condition table by function

def imgIndex_er(): 
    os.chdir(image_path)

    def f_lazer(z):
        start = z.find('_position')-3
        end = z.find('_position')
        return(z[start:end])

    def f_Position_ID(z):
        start = z.find('_position')+9 #Note: The shift forward is dependant upon how you write out position
        end = z.find('_time')   #Make sure that the 'n' is present in the array to confirm no place 0 has been lost
        return(z[start:end])

    def f_Frame(z):
        start = z.find('_time')+5
        end = z.find('_time')+9
        return(z[start:end])

    def f_expdate(x):
        dstart = x.find('_hr')-4
        dend = x.find('_hr')
        expdate = x[dstart:dend]
        expdate = expdate.translate({ord('-'): None})
        expdate = "d" + expdate
        return(expdate)

    def f_runnumb(d):
        rsta = d.find("hr")+2
        ren = d.find("hr")+6
        return(d[rsta:ren])

    count = 0
    imgIndex = []
    for root, dirs, files in os.walk(os.getcwd()):
        for name in files:
            if name.endswith(".tif"):
                imgIndex.append({'Path': os.path.join(root, name)})
                count = count + 1
                print(count, end="\r")
            else :
                pass
            
    imgIndex = pd.DataFrame(imgIndex)
    imgIndex['Date'] = pd.Series(imgIndex.iloc[:,0]).apply(f_expdate)
    imgIndex['PositionID'] = pd.Series(imgIndex.iloc[:,0]).apply(f_Position_ID) 
    imgIndex['Frame'] = pd.Series(imgIndex.iloc[:,0]).apply(f_Frame)
    imgIndex['Channel'] = pd.Series(imgIndex.iloc[:,0]).apply(f_lazer)
    imgIndex['Hour/Run_t'] =pd.Series(imgIndex.iloc[:,0]).apply(f_runnumb)
    #imgIndex["temp_date_hour"] = imgIndex["Date"] + imgIndex["Hour/Run_t"]
    imgIndex["Hour/Run_t"] = imgIndex["Hour/Run_t"].apply(pd.to_numeric)

    z = imgIndex[["Date", "Hour/Run_t"]]
    z = z.drop_duplicates().reset_index()

    z["Run"] = 0
    z["for"] = 1
    z = z.sort_values(by = ["Date", "Hour/Run_t"], ascending = True)

    for q in range(len(z)):
        r = 1
        if z.iloc[q,:]['for'] == 1:
            if z.iloc[q,1] == z.iloc[(q-1),1]:
                z.iloc[q,3] = (z.iloc[q-1,3] + 1)
                z.iloc[q,4] = 0
            else:
                z.iloc[q,3] = r
                z.iloc[q,4] = 0
        else:
            pass

    z = z[["index", "Date", "Run"]] #only grab the columns which are usefull, and not the ones created just for the logical loops

    t = len(z) - 1
    imgIndex ["Run"] = 0
    for g in range(len(z)):
        if g == 0:
            i1 = 0
            i2 = int(z.loc[g, ["index"]]) - 1
            imgIndex.loc[i1:i2, ["Run"]]  = z.iloc[g, 2]
        if g == t:
            i1 = int(z.loc[g, ["index"]])
            i2 = len(imgIndex) - 1
            imgIndex.loc[i1:i2, ["Run"]]  = z.iloc[g, 2]
        else:
            g2 = g + 1
            i1 = int(z.loc[g, ["index"]])
            i2 = int(z.loc[g2, ["index"]]) -1
            imgIndex.loc[i1:i2, ["Run"]]  = z.iloc[g, 2]
    del z

    del count
    del dirs
    del files
    del name

    imgIndex["Run"] = imgIndex["Run"].astype('str')
    imgIndex["Hour/Run_t"] = imgIndex["Hour/Run_t"].astype("str")
    imgIndex["Unique_pos"] = imgIndex['Date'] +"r" + imgIndex["Run"] + "p" + imgIndex['PositionID']
    imgIndex["Unique_pos_hour"] = imgIndex['Date'] + "h" + imgIndex["Hour/Run_t"] + "p" + imgIndex['PositionID']
    imgIndex["Unique_frame"] = imgIndex["Unique_pos"] + "f" + imgIndex['Frame'] #By performing this way, can avoid one extra step per index
    imgIndex.set_index(["Unique_frame"], inplace = True)
    imgIndex

    os.chdir(microfluidics_results)
    series = imgIndex["Unique_pos"]
    series = pd.unique(series)
    l = len(series)

    with open('Positions.txt', 'a+') as posit:
        for p in series:
            posit.write(f'{p} ')
    posit.close()

    imgIndex.to_csv("imgIndex.csv")
    return(imgIndex)
    
imgIndex = imgIndex_er()

In [11]:
imgIndex["Path"].values[0]

'D:\\Microfluidics\\Missing_Analyze\\20180224_hr0943_MMS\\20180224_hr0943_MMS\\2018-02-24_10-01-30\\GFP_position050200_time0001.tif'

In [12]:
#Updatae MASTER with elements of imgIndex
# MASTER = imgIndex
# JsonMast = MASTER.to_json(orient = 'split'))
# print(JsonMast)
# del MASTER

# os.chdir(microfluidics_results)

# if Experimental_info["data_subset"] == True:
#     subset_imgIndex = imgIndex[imgIndex["Unique_pos"].isin(instances)]
#     # subset_imgIndex = subset_imgIndex

#     imgIndex.to_hdf('MASTER.h5', key= 'long_imgIndex', mode='w')
#     subset_imgIndex.to_hdf('MASTER.h5', key= 'imgIndex', mode='r+')
    
#     imgIndex.to_csv("long_imgIndex.csv")
#     subset_imgIndex.to_csv('imgIndex.csv')

#     imgIndex = subset_imgIndex
#     del subset_imgIndex
# else:
#     imgIndex.to_hdf('MASTER.h5', key= 'imgIndex', mode='w')
#     imgIndex.to_csv("imgIndex.csv")

#Files must be saved and removed from ram to allow pipeline to run more efficiently
#This box works. If there is an error it is just because it has already been run
#10-12-2021: In currently implementation, files are read from csv because it is easier to access and not save over 

ImpPix = imgIndex[["Path", "Unique_frame", "Lazer", "Unique_pos"]]
ImpPix.set_index(['Unique_pos', "Unique_frame"], inplace = True)

ImpPix

mKO_only = imgIndex[imgIndex['Channel'] == "mKO"]
mKate_only = imgIndex[imgIndex['Channel'] == "mKa"]
GFP_only = imgIndex[imgIndex['Channel'] == "GFP"]

mKO_only.to_hdf('MASTER.h5', key= 'mKO_only', mode='r+')
mKate_only.to_hdf('MASTER.h5', key= 'mKa_only', mode='r+')
GFP_only.to_hdf('MASTER.h5', key= 'GFP_only', mode='r+')

del mKO_only
del mKate_only
del GFP_only

In [13]:
# BF_Sub_only = imgIndex[imgIndex["Channel"] == "Sub"]

# try: imgIndex
# except NameError: pass
# else: del imgIndex

# try: subet_imgIndex
# except NameError: pass
# else: del subset_imgIndex


#This is not currently being used. It was initial written for the segmentation automation, but was replaced with a more eficient xml creation and snakemake pipeline

In [14]:
# BF_Sub_only

In [15]:
# def oof_is_right(stri):
#     ooooof = (f"^Sub_position{stri}.*?\.tif")
#     return(ooooof)

# BF_Sub_only = BF_Sub_only[["Path", "Unique_pos", "PositionID"]]
# BF_Sub_only["regex"] = pd.Series(BF_Sub_only.iloc[:,2]).apply(oof_is_right)

In [16]:
# BF_Sub_only.drop_duplicates()

In [17]:
# os.chdir(microfluidics_results)
# BF_Sub_only.to_csv("BF_Sub_only.csv")
# del BF_Sub_only

In [18]:
#Create XML documents
# os.chdir(code) import XML_doc_maker # This will also create the results folders to which the documents will point

os.chdir(AllPoint)

count = 0
Series_series= []
for root, dirs, files in os.walk(os.getcwd()):
    for name in files:
        if name.endswith(".xml") and not name.startswith("PARAMS"):
            Series_series.append({'Name': os.path.join(name)})
            count = count + 1
            print(count, end="\r")
        else :
            pass
Series_series

In [19]:
##If running on linux, replace the windows the subprocess.run with <
##subprocess.run(CellX.sh MCR DIR/v715 PARAMS.XML -s SERIES.XML) #Linux>

#IN THIS PIPLELINE, I HAVE CHOSEN TO USE ONLY THE CELL TRACKING FUNCTIONALITY WILL NOT BE ALLOWING PROGRAM TO CALCULATE FLUORESCENT INTESTSITIES

# for ser in Series_series:
#     subprocess.run(f"{CellXc} PARAMS.xml -s {ser}")

# try: Series_series
# except NameError: pass
# else: del Series_series

In [20]:
#def G1():
#     with concurrent.futures.ProcessPoolExecutor as executor:
#         executor.submit(f_Position_ID, imgIndex.iloc[:,0])
# if __name__ == '__main__':
#     G1

In [21]:
microfluidics_results

'D:/Microfluidics/Missing_RESULTS'

In [22]:
# mask_folder = "C:/Users/Peter/Desktop/DATA_TEST/20180210/position740200_CellXOutput/GFP"
# os.chdir(mask_folder)

# TempTest = str(f"{microfluidics_results}/1TemTest")


############### This Allmasks index refers to those that are directly generated by the segmentation. The numbers in the masks are dependent the x position within the frame.


# os.chdir(TempTest)
def orgmask_er(): # All inputs are global variables
	os.chdir(microfluidics_results)

	orgmaskpaths = []
	count = 0
	fluors = "GFP_mKO_mKa"
	exclude = list(['No_fluor', 'No_flour', 'mKO_mKa'])
	for root, dirs, files, in os.walk(os.getcwd(), topdown= True):
		[dirs.remove(d) for d in list(dirs) if d in exclude]
		for name in files:
			if name.endswith(".mat") and name.startswith("mask"):
				orgmaskpaths.append({'Path': os.path.join(root, name)})
				count = count + 1
				print(count, end="\r")
			else: 
				pass

	def f_Pos_mask(z):
		start = z.find('p') #Note: The shift forward is dependant upon how you write out position
		end = z.find('GFP')-1  #Make sure that the 'n' is present in the array to confirm no place 0 has been lost
		pos = z[start:end]
		# if len(pos) <= 8:
		#     return(pos)
		# else:
		#     pass
		return(pos)

	def mask_inf(m):
		num1 = m.find('ask_')+5
		num2 = m.find('.mat')
		num_ = ("f"+ m[num1:num2])
		return(num_)

	def mask_hour(m):
		hourS = m.find("_")+2
		hourE = m.find("_p")
		return(m[hourS:hourE])

	def mask_barcode(m):
		start =m.find('d02')
		end = m.find('GFP') -1
		return (m[start:end])

	def f_maskdate(x):
		dstart = x.find('d0')
		dend = dstart + 5
		expdate = (x[dstart:dend])
		return(expdate)

	def Mod_epoch(f):
		t = os.path.getmtime(f)
		return(t)
		# return(time.ctime(t))

	orgAllmasks = pd.DataFrame(orgmaskpaths)
	def f_non_sync(p):
		substring = ".sync"
		if substring in p:
			return(None)
		else:
			return(p)
			
	orgAllmasks["Path"] = pd.Series(orgAllmasks.iloc[:,0]).apply(f_non_sync)
	orgAllmasks.dropna(inplace = True)
	orgAllmasks["Date"] = pd.Series(orgAllmasks.iloc[:,0]).apply(f_maskdate)
	orgAllmasks["Position_ID"] = pd.Series(orgAllmasks.iloc[:,0]).apply(f_Pos_mask)
	orgAllmasks["MaskNum"] = pd.Series(orgAllmasks.iloc[:,0]).apply(mask_inf)
	# Allmasks["Hour/Run_t"] = pd.Series(Allmasks.iloc[:,0]).apply(mask_hour)
	# Allmasks["Unique_pos_hour"] = Allmasks['Date'] + "h" + Allmasks["Hour/Run_t"] + Allmasks['Position_ID']

	orgAllmasks["Unique_pos"] = pd.Series(orgAllmasks.iloc[:,0]).apply(mask_barcode)
	orgAllmasks["Unique_frame"] = orgAllmasks["Unique_pos"] + orgAllmasks['MaskNum'] #By performing this way, can avoid one extra step per index
	orgAllmasks["Mod_epoch_org"] = pd.Series(orgAllmasks["Path"]).apply(Mod_epoch)
	orgAllmasks["Mod_date_org"] = pd.Series(orgAllmasks["Mod_epoch_org"]).apply(time.ctime)

	del orgmaskpaths

	orgAllmasks.dropna(inplace=True) # temp drop because there are some extra segmentations
	orgAllmasks.reset_index(inplace=True, drop = True)
	orgAllmasks.set_index(["Unique_frame"], inplace = True)
	orgAllmasks.to_csv("orgAllmasks.csv")

orgmask_er()

In [8]:
# mask_folder = "C:/Users/Peter/Desktop/DATA_TEST/20180210/position740200_CellXOutput/GFP"
# os.chdir(mask_folder)

# TempTest = str(f"{microfluidics_results}/1TemTest")


############### This Allmasks index refers to those that are generated after the tracking program has been run. The numbers in the masks have been corrected and are not dependent the x position.

# os.chdir(TempTest)
def Allmask_er():
	os.chdir(microfluidics_results)

	maskpaths = []
	count = 0
	for root, dirs, files, in os.walk(os.getcwd()):
		for name in files:
			if name.endswith(".mat") and name.startswith("track_mask"):
				maskpaths.append({'Path': os.path.join(root, name)})
				count = count + 1
				print(count, end="\r")
			else:
				pass

	# def f_Pos_mask(z):
	#     start = z.find('RESULTS') + 8 #Note: The shift forward is dependant upon how you write out position
	#     end = z.find('GFP')-1  #Make sure that the 'n' is present in the array to confirm no place 0 has been lost
	#     pos = z[start:end]
	#     if len(pos) <= 16:
	#         return(pos)
	#     else:
	#         pass

	def mask_inf(m):
		num1 = m.find('mask_')+6 #Made the number of zeros shorter. Should make sure the z fill is smaller in original file
		num2 = m.find('.mat')
		num_ = ("f"+ m[num1:num2])
		return(num_)

	def mask_hour(m):
		hourS = m.find("_")+2
		hourE = m.find("_p")
		return(m[hourS:hourE])

	def mask_barcode(m):
		start =m.find('d02')
		end = m.find('GFP') -1
		return (m[start:end])

	def f_maskdate(x):
		dstart = x.find('d0')
		dend = dstart + 5
		expdate = (x[dstart:dend])
		return(expdate)

	def Mod_epoch(f):
		t = os.path.getmtime(f)
		return(t)
		# return(time.ctime(t))
	def f_maskdate(x):
		dstart = x.find('d0')
		dend = dstart + 5
		expdate = (x[dstart:dend])
		return(expdate)

	Allmasks = pd.DataFrame(maskpaths)
	def f_non_sync(p):
		substring = ".sync"
		if substring in p:
			return(None)
		else:
			return(p)
			
	Allmasks["Path"] = pd.Series(Allmasks.iloc[:,0]).apply(f_non_sync)
	Allmasks.dropna(inplace = True)
	Allmasks["Date"] = pd.Series(Allmasks.iloc[:,0]).apply(f_maskdate)
	# Allmasks["Position_ID"] = pd.Series(Allmasks.iloc[:,0]).apply(f_Pos_mask)
	Allmasks["MaskNum"] = pd.Series(Allmasks.iloc[:,0]).apply(mask_inf)
	# Allmasks["Hour/Run_t"] = pd.Series(Allmasks.iloc[:,0]).apply(mask_hour)
	# Allmasks["Unique_pos_hour"] = Allmasks['Date'] + "h" + Allmasks["Hour/Run_t"] + Allmasks['Position_ID']

	Allmasks["Unique_pos"] = pd.Series(Allmasks.iloc[:,0]).apply(mask_barcode)
	Allmasks["Unique_frame"] = Allmasks["Unique_pos"] + Allmasks['MaskNum'] #By performing this way, can avoid one extra step per index
	Allmasks["Mod_epoch_tM"] = pd.Series(Allmasks["Path"]).apply(Mod_epoch)
	Allmasks["Mod_date_tM"] = pd.Series(Allmasks["Mod_epoch_tM"]).apply(time.ctime)


	# del maskpaths

	# Allmasks.dropna(inplace=True) # temp drop because there are some extra segmentations
	Allmasks.reset_index(inplace=True, drop = True)
	Allmasks.to_csv("Allmasks.csv", index = False)
	return(Allmasks)

am = Allmask_er()
am

,Path,Date,MaskNum,Unique_pos,Unique_frame,Mod_epoch_tM,Mod_date_tM
0,F:\Microfluidics\RESULTS\d0209r1p020100\GFP_mK...,d0209,f0001,d0209r1p020100,d0209r1p020100f0001,1.647693e+09,Sat Mar 19 08:32:17 2022
1,F:\Microfluidics\RESULTS\d0209r1p020100\GFP_mK...,d0209,f0002,d0209r1p020100,d0209r1p020100f0002,1.647693e+09,Sat Mar 19 08:32:31 2022
2,F:\Microfluidics\RESULTS\d0209r1p020100\GFP_mK...,d0209,f0003,d0209r1p020100,d0209r1p020100f0003,1.647693e+09,Sat Mar 19 08:32:37 2022
3,F:\Microfluidics\RESULTS\d0209r1p020100\GFP_mK...,d0209,f0004,d0209r1p020100,d0209r1p020100f0004,1.647693e+09,Sat Mar 19 08:32:42 2022
4,F:\Microfluidics\RESULTS\d0209r1p020100\GFP_mK...,d0209,f0005,d0209r1p020100,d0209r1p020100f0005,1.647693e+09,Sat Mar 19 08:32:48 2022
...,...,...,...,...,...,...,...
45767,F:\Microfluidics\RESULTS\d0224r1p990200\GFP_mK...,d0224,f0062,d0224r1p990200,d0224r1p990200f0062,1.656381e+09,Mon Jun 27 21:46:01 2022
45768,F:\Microfluidics\RESULTS\d0224r1p990200\GFP_mK...,d0224,f0063,d0224r1p990200,d0224r1p990200f0063,1.656381e+09,Mon Jun 27 21:46:04 2022
45769,F:\Microfluidics\RESULTS\d0224r1p990200\GFP_mK...,d0224,f0064,d0224r1p990200,d0224r1p990200f0064,1.656381e+09,Mon Jun 27 21:46:07 2022
45770,F:\Microfluidics\RESULTS\d0224r1p990200\GFP_mK...,d0224,f0065,d0224r1p990200,d0224r1p990200f0065,1.656381e+09,Mon Jun 27 21:46:11 2022


In [9]:
def expMask_er():
	os.chdir(microfluidics_results)
	maskpaths = []
	count = 0
	for root, dirs, files, in os.walk(os.getcwd()):
		for name in files:
			if name.endswith(".mat") and name.startswith("frame_pxMAT"):
				maskpaths.append({'Path': os.path.join(root, name)})
				count = count + 1
				print(count, end="\r")
			else:
				pass

	def f_Pos_mask(z):
		start = z.find('RESULTS') + 10 #Note: The shift forward is dependant upon how you write out position
		end = z.find('GFP_')-1  #Make sure that the 'n' is present in the array to confirm no place 0 has been lost
		pos = z[start:end]
		if len(pos) <= 17:
			return(pos)
		else:
			pass

	def mask_num(m):
		num1 = m.find('pxMAT')+7
		num2 = m.find('.mat')
		num_ = ("f"+ m[num1:num2])
		return(num_)

	def Mod_epoch(f):
		t = os.path.getmtime(f)
		return(t)
		# return(time.ctime(t))

	def f_maskdate(x):
		dstart = x.find('d0')
		dend = dstart + 5
		expdate = (x[dstart:dend])
		return(expdate)

	def mask_barcode(m):
		start =m.find('d02')
		end = m.find('GFP') -1
		return (m[start:end])

	Allmasks_exp = pd.DataFrame(maskpaths)
	def f_non_sync(p):
		substring = ".sync"
		if substring in p:
			return(None)
		else:
			return(p)
			
	Allmasks_exp["Path"] = pd.Series(Allmasks_exp.iloc[:,0]).apply(f_non_sync)
	Allmasks_exp.dropna(inplace = True )
	Allmasks_exp["Date"] = pd.Series(Allmasks_exp.iloc[:,0]).apply(f_maskdate)
	# Allmasks_exp["Position_ID"] = pd.Series(Allmasks_exp.iloc[:,0]).apply(f_Pos_mask)
	Allmasks_exp["MaskNum"] = pd.Series(Allmasks_exp.iloc[:,0]).apply(mask_num)
	# Allmasks_exp["Hour/Run_t"] = pd.Series(Allmasks_exp.iloc[:,0]).apply(mask_hour)
	# Allmasks_exp["Unique_pos_hour"] = Allmasks_exp['Date'] + "h" + Allmasks_exp["Hour/Run_t"] + Allmasks_exp['Position_ID']

	Allmasks_exp["Unique_pos"] = pd.Series(Allmasks_exp.iloc[:,0]).apply(mask_barcode)
	Allmasks_exp["DayRun"] = pd.Series(Allmasks_exp["Unique_pos"]).apply(lambda x: x[:x.find("p")])
	Allmasks_exp["Unique_frame"] = Allmasks_exp["Unique_pos"] + Allmasks_exp['MaskNum'] #By performing this way, can avoid one extra step per index
	Allmasks_exp["Mod_epoch_exp"] = pd.Series(Allmasks_exp["Path"]).apply(Mod_epoch)
	Allmasks_exp["Mod_date_exp"] = pd.Series(Allmasks_exp["Mod_epoch_exp"]).apply(time.ctime)

	Allmasks_exp.set_index(["Unique_frame"], inplace = True)
	# Allmasks_exp = Allmasks_exp[Allmasks_exp["Date"] == 'd0224']

	Allmasks_exp.to_csv('Allmasks_exp.csv')
	return(Allmasks_exp)
	
x = expMask_er()
x

,Path,Date,MaskNum,Unique_pos,DayRun,Mod_epoch_exp,Mod_date_exp
Unique_frame,,,,,,,
d0209r1p020100f0001,F:\Microfluidics\RESULTS\d0209r1p020100\GFP_mK...,d0209,f0001,d0209r1p020100,d0209r1,1.655147e+09,Mon Jun 13 15:11:38 2022
d0209r1p020100f0002,F:\Microfluidics\RESULTS\d0209r1p020100\GFP_mK...,d0209,f0002,d0209r1p020100,d0209r1,1.655147e+09,Mon Jun 13 15:11:38 2022
d0209r1p020100f0003,F:\Microfluidics\RESULTS\d0209r1p020100\GFP_mK...,d0209,f0003,d0209r1p020100,d0209r1,1.655147e+09,Mon Jun 13 15:11:38 2022
d0209r1p020100f0004,F:\Microfluidics\RESULTS\d0209r1p020100\GFP_mK...,d0209,f0004,d0209r1p020100,d0209r1,1.655147e+09,Mon Jun 13 15:11:38 2022
d0209r1p020100f0005,F:\Microfluidics\RESULTS\d0209r1p020100\GFP_mK...,d0209,f0005,d0209r1p020100,d0209r1,1.655147e+09,Mon Jun 13 15:11:38 2022
...,...,...,...,...,...,...,...
d0224r1p150200f0020,F:\Microfluidics\RESULTS\d0224r1p150200\GFP_mK...,d0224,f0020,d0224r1p150200,d0224r1,1.658181e+09,Mon Jul 18 17:53:09 2022
d0224r1p150200f0021,F:\Microfluidics\RESULTS\d0224r1p150200\GFP_mK...,d0224,f0021,d0224r1p150200,d0224r1,1.658181e+09,Mon Jul 18 17:53:12 2022
d0224r1p150200f0022,F:\Microfluidics\RESULTS\d0224r1p150200\GFP_mK...,d0224,f0022,d0224r1p150200,d0224r1,1.658181e+09,Mon Jul 18 17:53:13 2022


In [ ]:
x

,Path,Date,MaskNum,Unique_pos,DayRun,Mod_epoch_exp,Mod_date_exp
Unique_frame,,,,,,,
d0209r1p020100f0001,D:\Microfluidics\Missing_RESULTS2\d0209r1p0201...,d0209,f0001,d0209r1p020100,d0209r1,1.655147e+09,Mon Jun 13 15:11:38 2022
d0209r1p020100f0002,D:\Microfluidics\Missing_RESULTS2\d0209r1p0201...,d0209,f0002,d0209r1p020100,d0209r1,1.655147e+09,Mon Jun 13 15:11:38 2022
d0209r1p020100f0003,D:\Microfluidics\Missing_RESULTS2\d0209r1p0201...,d0209,f0003,d0209r1p020100,d0209r1,1.655147e+09,Mon Jun 13 15:11:38 2022
d0209r1p020100f0004,D:\Microfluidics\Missing_RESULTS2\d0209r1p0201...,d0209,f0004,d0209r1p020100,d0209r1,1.655147e+09,Mon Jun 13 15:11:38 2022
d0209r1p020100f0005,D:\Microfluidics\Missing_RESULTS2\d0209r1p0201...,d0209,f0005,d0209r1p020100,d0209r1,1.655147e+09,Mon Jun 13 15:11:38 2022
...,...,...,...,...,...,...,...
d0220r1p470300f0041,D:\Microfluidics\Missing_RESULTS2\d0220r1p4703...,d0220,f0041,d0220r1p470300,d0220r1,1.655242e+09,Tue Jun 14 17:26:48 2022
d0220r1p470300f0042,D:\Microfluidics\Missing_RESULTS2\d0220r1p4703...,d0220,f0042,d0220r1p470300,d0220r1,1.655242e+09,Tue Jun 14 17:27:02 2022
d0220r1p470300f0043,D:\Microfluidics\Missing_RESULTS2\d0220r1p4703...,d0220,f0043,d0220r1p470300,d0220r1,1.655242e+09,Tue Jun 14 17:27:12 2022


In [ ]:
microfluidics_results

'D:/Microfluidics/Missing_RESULTS'

In [ ]:
Allmasks.sort_values(by = "Unique_frame", inplace= True)

In [ ]:
# os.chdir(microfluidics_results)
# try: imgIndex = pd.read_hdf('MASTER.h5', 'long_imgIndex')
# except KeyError: imgIndex = pd.read_hdf('MASTER.h5', 'imgIndex')
# else: imgIndex = pd.read_hdf('MASTER.h5', 'long_imgIndex')
# imgIndex

In [ ]:
# u = imgIndex.drop_duplicates()
# u = u.set_index(["Unique_pos_hour"])

# def run_match(q):
#     run_num = u.loc[[q],["Run"]]
#     run_num = run_num.drop_duplicates()
#     return(run_num.iloc[0,0])

# Allmasks["Run"] = pd.Series(Allmasks.iloc[:,5]).apply(run_match)

# try: u
# except NameError: pass
# else: del u

# del imgIndex

In [ ]:
# Allmasks["Unique_pos"] = Allmasks['Date'] +"r" + Allmasks["Run"] + Allmasks['Position_ID']
# Allmasks["Unique_frame"] = Allmasks["Unique_pos"] + Allmasks['MaskNum'] #By performing this way, can avoid one extra step per index

,Path,Date,Position_ID,MaskNum,Unique_pos
Unique_frame,,,,,
d0224r1p1050200f0064,D:\Microfluidics\Missing_RESULTS\.sync\Archive...,d0224,p1050200,f0064,d0224r1p1050200
d0223r1p070300f0063,D:\Microfluidics\Missing_RESULTS\d0223r1p07030...,d0223,p070300,f0063,d0223r1p070300
d0223r1p070300f0061,D:\Microfluidics\Missing_RESULTS\d0223r1p07030...,d0223,p070300,f0061,d0223r1p070300
d0223r1p070300f0062,D:\Microfluidics\Missing_RESULTS\d0223r1p07030...,d0223,p070300,f0062,d0223r1p070300
d0223r1p070300f0059,D:\Microfluidics\Missing_RESULTS\d0223r1p07030...,d0223,p070300,f0059,d0223r1p070300
...,...,...,...,...,...
d0212r1p140200f0006,D:\Microfluidics\Missing_RESULTS\d0212r1p14020...,d0212,p140200,f0006,d0212r1p140200
d0212r1p140200f0008,D:\Microfluidics\Missing_RESULTS\d0212r1p14020...,d0212,p140200,f0008,d0212r1p140200
d0212r1p140200f0001,D:\Microfluidics\Missing_RESULTS\d0212r1p14020...,d0212,p140200,f0001,d0212r1p140200


In [ ]:
Allmasks.set_index(["Unique_frame"], inplace = True)
# Allmasks = Allmasks[Allmasks["Unique_pos"].isin(instances)]

In [ ]:
#Inefficient and cannot be stored properly

##Mask combinining based on cell. Store matices in sparse form, generating a raw matrix from the file, then a simplified version to store what is cell
def mask_load(m):
    mask = loadmat(m)
    a = mask['segmentationMask']
    small = a
    return (small)

def simp_mask_load(m):
    mask = loadmat(m)
    simp = mask['segmentationMask']
    simp[simp >=1 ] = 1
    small = simp
    return (small)

# def sparse_mask_load(m):
#     mask = loadmat(m)
#     a = mask['segmentationMask']
#     small = sp.csr_matrix(a)
#     return (small)

# def simp_sparse_mask_load(m):
#     mask = loadmat(m)
#     simp = mask['segmentationMask']
#     simp[simp >=1 ] = 1
#     small = sp.csr_matrix(simp)
#     return (small)

#Allmasks["mask_r"] = pd.Series(Allmasks.iloc[:,0]).apply(sparse_mask_load)
#Allmasks["mask_s"] = pd.Series(Allmasks.iloc[:,0]).apply(simp_sparse_mask_load)



In [ ]:
# yes = ["d0218r2p900200"]
# Allmasks = Allmasks[Allmasks["Unique_pos"].isin(yes)]
# Allmasks.to_csv('Allmasks.csv')


In [ ]:
#For now, subset the data to a day so that it is more manageable
# Allmasks = Allmasks[(Allmasks["Date"] == 'd0219') | (Allmasks["Date"] == 'd0218')] #Subset for the 18th and 19th
# Allmasks = Allmasks[(Allmasks["Date"] == 'd0222') & (Allmasks["Unique_pos"].isin(instances))]
Allmasks = Allmasks[Allmasks["Unique_pos"].isin(instances)]

In [ ]:
#Save mask index files to csv files
# orgAllmasks.to_csv('orgAllmasks.csv')
Allmasks.to_csv('Allmasks.csv')

In [ ]:
os.getcwd()

'D:\\Microfluidics\\Missing_RESULTS'

In [ ]:
os.chdir(microfluidics_results)


# Save files into hdf form. This will be reimplemented for final run so that a single file can be moved from server with all information

# if Experimental_info["data_subset"] == True:
#     subset_Allmasks = Allmasks[Allmasks["Unique_pos"].isin(instances)]
#     subset_Allmasks = subset_Allmasks

#     Allmasks.to_hdf('MASTER.h5', key= 'long_orgAllmasks', mode='r+')
#     subset_Allmasks.to_hdf('MASTER.h5', key= 'orgAllmasks', mode='r+')

#     Allmasks.to_csv('long_orgAllmasks.csv')
#     subset_Allmasks.to_csv("orgAllmasks.csv")

#     Allmasks = subset_Allmasks
#     del subset_Allmasks
# else:
#     Allmasks.to_hdf('MASTER.h5', key= 'orgAllmasks', mode='r+')

# if Experimental_info["data_subset"] == True:
#     subset_Allmasks = Allmasks[Allmasks["Unique_pos"].isin(instances)]
#     subset_Allmasks = subset_Allmasks

#     Allmasks.to_hdf('MASTER.h5', key= 'long_Allmasks', mode='r+')
#     subset_Allmasks.to_hdf('MASTER.h5', key= 'Allmasks', mode='r+')

#     Allmasks.to_csv('long_Allmasks.csv')
#     subset_Allmasks.to_csv("Allmasks.csv")

#     Allmasks = subset_Allmasks
#     del subset_Allmasks
# else:
#     Allmasks.to_hdf('MASTER.h5', key= 'Allmasks', mode='r+')

In [ ]:
# #Set the fluorescent dictionary which will be used to relate image folders to conditions
# # Move the condition information into the common HDF file for easier access
# Condition_information = pd.read_excel(locationC)
# Condition_information = Condition_information.dropna()
# os.chdir(microfluidics_results)
# Condition_information.to_hdf('MASTER.h5', key= 'Condition_information', mode='r+')
# Condition_information.to_csv('Condition_information.csv')

In [ ]:
# os.chdir(microfluidics_results)
# Condition_information = pd.read_hdf("MASTER.h5", 'Condition_informaton')

In [ ]:
# os.chdir(microfluidics_results)
# imgIndex = pd.read_hdf('MASTER.h5', key = 'imgIndex')
# imgIndex

In [33]:
os.chdir(microfluidics_results)
def info_index_er():
    count = 0
    info_index = []
    #This loop should find all the image paths for all the days
    for root, dirs, files, in os.walk(os.getcwd()):
        for name in files:
            if name.endswith(".txt") and name.startswith("TrackingResults", 13):
                info_index.append({'Path': os.path.join(root, name)})
                count = count + 1
                print(count, end="\r")
            elif name.endswith(".txt") and name.startswith("TrackingResults", 14):
                info_index.append({'Path': os.path.join(root, name)})
                count = count + 1
                print(count, end="\r")
            elif name.endswith(".txt") and name.startswith("TrackingResults", 15): #if name.endswith(".txt") and name.startswith("cells"): #changed to 15
                info_index.append({'Path': os.path.join(root, name)})
                count = count + 1
                print(count, end="\r")
            elif name.endswith(".txt") and name.startswith("TrackingResults", 16): #if name.endswith(".txt") and name.startswith("cells"): #changed to 15
                info_index.append({'Path': os.path.join(root, name)})
                count = count + 1
                print(count, end="\r")
            else:
                pass
    del count

    def sync_rem(x):
        if x.startswith(".sync"):
            return(None)
        else:
            return(x)

    def Pos_label(t):
        pstart = t.find("RESULTS")+8 #9 # It is 9 here because "MissingRESULTS2"
        #8 # Must change if "RES_N_ULTS" and to +11
        pend = t.find('GFP_mKO_mKa')-1
        return(t[pstart:pend])

    def Budneck(t):
        bstart = t.find('GFP_mKO_mKa') + 12
        bend = t.find('TrackingResults') - 16
        return(t[bstart:bend])

    def Mod_epoch_time(f):
        return(os.path.getmtime(f))
        

    def f_non_sync(p):
        substring = ".sync"
        if substring in p:
            return(None)
        else:
            return(p)
            
    
    info_index = pd.DataFrame(info_index)
    info_index["Path"] = pd.Series(info_index.iloc[:,0]).apply(f_non_sync)
    info_index.dropna(inplace= True)

    info_index["Pos"] = pd.Series(info_index.iloc[:,0]).apply(Pos_label)
    info_index["Mod_epoch_info"] = pd.Series(info_index.iloc[:,0]).apply(Mod_epoch_time)
    info_index = info_index.loc[info_index.groupby("Pos")["Mod_epoch_info"].idxmax()]
    info_index["Mod_date_info"] = pd.Series(info_index["Mod_epoch_info"]).apply(time.ctime)

    # info_index.to_hdf('MASTER.h5', key= 'info_index', mode='r+')

    info_index.to_csv('info_index.csv', index = False)
    return(info_index)

info_index = info_index_er()

In [34]:
info_index[info_index["Pos"] == 'd0212r1p020200']

,Path,Pos,Mod_epoch_info,Mod_date_info
112,D:\Microfluidics\Missing_RESULTS\d0212r1p02020...,d0212r1p020200,1.659909e+09,Sun Aug 7 17:48:58 2022


In [ ]:
count = 0
info_index = []
#This loop should find all the image paths for all the days
for root, dirs, files, in os.walk(os.getcwd()):
    for name in files:
        if name.endswith(".txt") and name.startswith("TrackingResults", 13):
            info_index.append({'Path': os.path.join(root, name)})
            count = count + 1
            print(count, end="\r")
        elif name.endswith(".txt") and name.startswith("TrackingResults", 14):
            info_index.append({'Path': os.path.join(root, name)})
            count = count + 1
            print(count, end="\r")
        elif name.endswith(".txt") and name.startswith("TrackingResults", 15): #if name.endswith(".txt") and name.startswith("cells"): #changed to 15
            info_index.append({'Path': os.path.join(root, name)})
            count = count + 1
            print(count, end="\r")
        elif name.endswith(".txt") and name.startswith("TrackingResults", 16): #if name.endswith(".txt") and name.startswith("cells"): #changed to 15
            info_index.append({'Path': os.path.join(root, name)})
            count = count + 1
            print(count, end="\r")
        else:
            pass
del count

def sync_rem(x):
    if x.startswith(".sync"):
        return(None)
    else:
        return(x)

def Pos_label(t):
    pstart = t.find("RESULTS")+8 #9 # It is 9 here because "MissingRESULTS2"
    #8 # Must change if "RES_N_ULTS" and to +11
    pend = t.find('GFP_mKO_mKa')-1
    return(t[pstart:pend])

def Budneck(t):
    bstart = t.find('GFP_mKO_mKa') + 12
    bend = t.find('TrackingResults') - 16
    return(t[bstart:bend])

def Mod_epoch_time(f):
    return(os.path.getmtime(f))
    

def f_non_sync(p):
    substring = ".sync"
    if substring in p:
        return(None)
    else:
        return(p)
        

info_index = pd.DataFrame(info_index)
info_index["Path"] = pd.Series(info_index.iloc[:,0]).apply(f_non_sync)
info_index.dropna(inplace= True)

info_index["Pos"] = pd.Series(info_index.iloc[:,0]).apply(Pos_label)
info_index["Mod_epoch_info"] = pd.Series(info_index.iloc[:,0]).apply(Mod_epoch_time)
info_index = info_index.loc[info_index.groupby("Pos")["Mod_epoch_info"].idxmax()]
info_index["Mod_date_info"] = pd.Series(info_index["Mod_epoch_info"]).apply(time.ctime)

In [ ]:
info_index

,Path,Pos,Mod_epoch_info,Mod_date_info
88,D:\Microfluidics\Missing_RESULTS\d0210r1p02010...,d0210r1p020100,1.646789e+09,Tue Mar 8 20:19:24 2022
84,D:\Microfluidics\Missing_RESULTS\d0210r1p02020...,d0210r1p020200,1.646789e+09,Tue Mar 8 20:18:14 2022
87,D:\Microfluidics\Missing_RESULTS\d0210r1p02030...,d0210r1p020300,1.646789e+09,Tue Mar 8 20:17:48 2022
101,D:\Microfluidics\Missing_RESULTS\d0210r1p03010...,d0210r1p030100,1.646789e+09,Tue Mar 8 20:18:48 2022
102,D:\Microfluidics\Missing_RESULTS\d0210r1p03020...,d0210r1p030200,1.646789e+09,Tue Mar 8 20:21:08 2022
...,...,...,...,...
29,D:\Microfluidics\Missing_RESULTS\d0224r1p86030...,d0224r1p860300,1.656381e+09,Mon Jun 27 21:42:40 2022
21,D:\Microfluidics\Missing_RESULTS\d0224r1p89020...,d0224r1p890200,1.656381e+09,Mon Jun 27 21:42:52 2022
43,D:\Microfluidics\Missing_RESULTS\d0224r1p95020...,d0224r1p950200,1.656380e+09,Mon Jun 27 21:40:56 2022
22,D:\Microfluidics\Missing_RESULTS\d0224r1p96030...,d0224r1p960300,1.656381e+09,Mon Jun 27 21:42:46 2022


In [ ]:
# t sorted(glob('cells_0001*.csv'))
# All_chambers_df = pd.concat((pd.read_csv(file) for file in All_chambers_l), ignore_index= True)
# t

In [ ]:
# info_index["Path"]

In [ ]:
# info_index = info_index[info_index["Pos"].isin(instances)]

In [ ]:
info_index

,Path,Pos,Mod_epoch_info,Mod_date_info
0,D:\Microfluidics\Missing_RESULTS2\d0209r1p0201...,d0209r1p020100,1.647693e+09,Sat Mar 19 08:32:18 2022
1,D:\Microfluidics\Missing_RESULTS2\d0209r1p0202...,d0209r1p020200,1.647676e+09,Sat Mar 19 03:51:24 2022
3,D:\Microfluidics\Missing_RESULTS2\d0209r1p0203...,d0209r1p020300,1.647658e+09,Fri Mar 18 22:50:00 2022
4,D:\Microfluidics\Missing_RESULTS2\d0209r1p0301...,d0209r1p030100,1.647672e+09,Sat Mar 19 02:34:20 2022
6,D:\Microfluidics\Missing_RESULTS2\d0209r1p0302...,d0209r1p030200,1.647659e+09,Fri Mar 18 23:05:38 2022
7,D:\Microfluidics\Missing_RESULTS2\d0209r1p0303...,d0209r1p030300,1.647654e+09,Fri Mar 18 21:42:36 2022
9,D:\Microfluidics\Missing_RESULTS2\d0209r1p0601...,d0209r1p060100,1.647646e+09,Fri Mar 18 19:28:48 2022
10,D:\Microfluidics\Missing_RESULTS2\d0209r1p0602...,d0209r1p060200,1.647656e+09,Fri Mar 18 22:16:20 2022
11,D:\Microfluidics\Missing_RESULTS2\d0209r1p0603...,d0209r1p060300,1.647662e+09,Fri Mar 18 23:46:44 2022
12,D:\Microfluidics\Missing_RESULTS2\d0209r1p0701...,d0209r1p070100,1.647664e+09,Sat Mar 19 00:21:58 2022


In [ ]:
def info_concat_er():
    info_concat = pd.DataFrame([]) #this name is old but will contintue to be used until variable reassigment durring cleanup

    for p in range(2):
        g = info_index.iloc[p,0]
        x = pd.read_csv(g, sep = "\t", usecols = ['cell_frame', 'cell_index', 'cell_majoraxis', 'cell_minoraxis', 'cell_area', 'cell_volume', 'cell_perimeter', 'cell_eccentricity', 'cell_fractionOfGoodMembranePixels', 'cell_mem_area', 'cell_mem_volume', 'cell_nuc_radius', 'cell_nuc_area', 'cell_pole1_age', 'cell_pole2_age', 'cell_timepoint', 'track_index', 'track_fingerprint_binary_distance', 'track_fingerprint_real_distance', 'track_age', 'track_parent', 'track_parent_frame', 'track_parent_prob', 'track_parent_score', 'track_generation', 'track_lineage_tree', 'track_cell_cycle_phase', 'track_assignment_fraction', 'track_start_frame', 'track_end_frame', 'track_index_corrected', 'track_has_bud', 'track_budneck_total'])

        x['Unique_pos'] = info_index.iloc[p,1]
        info_concat = info_concat.append(x)

    info_concat["Unique_frame"] = info_concat["Unique_pos"] + "f" + info_concat["cell_frame"].astype(str).str.zfill(4)
    info_concat["Circ(major/minor)"] = info_concat["cell_majoraxis"]/info_concat["cell_minoraxis"]
    info_concat["Unique_cell"] = info_concat['Unique_pos'] + "c" + info_concat["track_index"].astype(str).str.zfill(4) ########### SUPER IMPORTANT. Correction to track_index from cell_index
    info_concat.set_index(['Unique_cell'], inplace = True)
    info_concat["Track_length"] = info_concat["track_end_frame"] - info_concat["track_start_frame"]

    info_concat["Circ(major/minor)"] = info_concat["cell_majoraxis"]/info_concat["cell_minoraxis"]
    info_concat["Track_length"] = info_concat["track_end_frame"] - info_concat["track_start_frame"]
    info_simple = info_concat.loc[:,["track_index", "Unique_frame", "track_start_frame", "Track_length", "cell_area", "cell_volume", "cell_perimeter", "Circ(major/minor)"]] # Further subset the file

    # too_short_drop = info_simple[info_simple["Track_length"] < 6].reset_index()["Unique_cell"] ## This was subsetting based on the
    # curdate = str(datetime.date.today())
    # too_short_drop.to_csv(f"Dropped_bc_short_{curdate}")
    # info_simple = info_simple[info_simple["Track_length"] >= 6] #Filter info to those which are present for more than 6 frames. For now, this has been set as the arbritrary lower limit of frames required to determine current localization.

    def pos(x):
        end = x.find("f")
        return(x[:end])

    info_simple["Unique_postion"] = pd.Series(info_simple["Unique_frame"]).apply(pos)
    info_concat.to_csv('info_concat.csv', index = False)
    info_simple.to_csv('info_simple.csv')
    return(info_simple)

q = info_concat_er()


# info_concat = pd.concat((pd.read_csv(file, sep = "\t", usecols = ['cell_frame', 'cell_index', 'cell_majoraxis', 'cell_minoraxis', 'cell_area', 'cell_volume', 'cell_perimeter', 'cell_eccentricity', 'cell_fractionOfGoodMembranePixels', 'cell_mem_area', 'cell_mem_volume', 'cell_nuc_radius', 'cell_nuc_area', 'cell_pole1_age', 'cell_pole2_age', 'cell_timepoint', 'track_index', 'track_fingerprint_binary_distance', 'track_fingerprint_real_distance', 'track_age', 'track_parent', 'track_parent_frame', 'track_parent_prob', 'track_parent_score', 'track_generation', 'track_lineage_tree', 'track_cell_cycle_phase', 'track_assignment_fraction', 'track_start_frame', 'track_end_frame', 'track_index_corrected', 'track_has_bud', 'track_budneck_total']) for file in info_index["Path"]), ignore_index= True) # This should be computationally more efficieint than above using the non-stored method. Also only reads in the columns that are useful to avoid future subsetting (RAM never tied up when not needed)

# # The pos info was not captured
# info_concat["Unique_frame"] = info_concat["Unique_pos"] + "f" + info_concat["cell_frame"].astype(str).str.zfill(4)
# info_concat["Circ(major/minor)"] = info_concat["cell_majoraxis"]/info_concat["cell_minoraxis"]
# info_concat["Unique_cell"] = info_concat['Unique_pos'] + "c" + info_concat["track_index"].astype(str).str.zfill(4) ########### SUPER IMPORTANT. Correction to track_index from cell_index
# info_concat.set_index(['Unique_cell'], inplace = True)
# info_concat["Track_length"] = info_concat["track_end_frame"] - info_concat["track_start_frame"]

# info_concat

In [ ]:
q

,track_index,Unique_frame,track_start_frame,Track_length,cell_area,cell_volume,cell_perimeter,Circ(major/minor),Unique_postion
0,1,d0213r1p020100f0001,1,101,441,6410,72,1.086957,d0213r1p020100
1,2,d0213r1p020100f0001,1,101,326,4240,61,1.050000,d0213r1p020100
2,3,d0213r1p020100f0001,1,101,504,7968,77,1.040000,d0213r1p020100
3,4,d0213r1p020100f0001,1,101,419,6020,70,1.090909,d0213r1p020100
4,5,d0213r1p020100f0001,1,19,144,1244,40,1.153846,d0213r1p020100
...,...,...,...,...,...,...,...,...,...
38763,418,d0213r1p020200f0102,54,48,929,16231,113,1.666667,d0213r1p020200
38764,419,d0213r1p020200f0102,54,48,958,17034,116,1.482759,d0213r1p020200
38765,455,d0213r1p020200f0102,57,45,1033,19775,123,1.387097,d0213r1p020200
38766,360,d0213r1p020200f0102,47,55,1099,18075,135,2.153846,d0213r1p020200


In [ ]:

info_simple

,track_index,Unique_frame,track_start_frame,Track_length,cell_area,cell_volume,cell_perimeter,Circ(major/minor),Unique_postion
0,1,d0213r1p020100f0001,1,101,441,6410,72,1.086957,d0213r1p020100
1,2,d0213r1p020100f0001,1,101,326,4240,61,1.050000,d0213r1p020100
2,3,d0213r1p020100f0001,1,101,504,7968,77,1.040000,d0213r1p020100
3,4,d0213r1p020100f0001,1,101,419,6020,70,1.090909,d0213r1p020100
4,5,d0213r1p020100f0001,1,19,144,1244,40,1.153846,d0213r1p020100
...,...,...,...,...,...,...,...,...,...
6176,237,d0222r2p970300f0069,59,10,599,7435,101,2.100000,d0222r2p970300
6177,211,d0222r2p970300f0069,50,19,956,16878,116,1.607143,d0222r2p970300
6178,217,d0222r2p970300f0069,51,18,1169,19597,139,2.400000,d0222r2p970300
6179,166,d0222r2p970300f0069,35,34,927,14939,112,1.769231,d0222r2p970300


In [ ]:
os.chdir(microfluidics_results)
# info_concat.to_hdf('MASTER.h5', key= 'info_all', mode='r+')
# info_simple.to_hdf('MASTER.h5', key= 'info_simp', mode='r+')
# os.chdir(microfluidics_results)


# try: info_concat
# except NameError: pass
# else: del info_concat

# try: info_simple
# except NameError: pass
# else: del info_simple

In [ ]:
# info_concat = pd.read_csv('info_concat.csv')
# info_concat.set_index(["Unique_cell"], inplace = True)
# info_concat

In [ ]:
os.chdir(microfluidics_results)
info_simp = pd.read_hdf('MASTER.h5', 'info_simp')

FileNotFoundError: File MASTER.h5 does not exist

In [ ]:
df1 = pd.DataFrame([{'a':1,'b':2},{'a':1,'b':3},{'a':2,'b':4}],
                   columns = ['a','b']).set_index('a')

df2 = pd.DataFrame([{'a':1,'c':5},{'a':2,'c':6}],columns = ['a','c']).set_index('a')
pd.concat([df1, df2], axis = 1)

,b,c
a,,
1,2,5
1,3,5
2,4,6


In [ ]:
df = pd.DataFrame([{"a":"d0213_1833_p020300f0001c0001", "b":1}, {"a":"d0213_1833_p020300f0001c0001", "b": 2}, {"a": "d0213_1833_p020300f0001c0002", "b":1}, {"a": "d0213_1833_p020300f0001c0002", "b":2}], columns = ["a", "b"]).set_index("Unique_cell")
df2 = pd.DataFrame([{"a": "d0213_1833_p020300f0001c0001", "c":2 }, {"a": "d0213_1833_p020300f0001c0001", "c":2}], columns = ["a", "c"])

p = info_concat.loc[("d0213_1833_p020300f0001c0001", "d0213_1833_p020300f0001c0002"), :]
p.reset_index(inplace = True)
p 
df.merge(p, how = "left"= "a", right_on = "Unique_cell")
# df = pd.concat([df,df2], axis = 1)
# df["cell_index", "cell_area", "cell_volume", "cell_perimeter"] = 


SyntaxError: invalid syntax (<ipython-input-81-ad0d1ad3d199>, line 7)

for n in range(len(GFP_layer)):
    x = GFP_layer[0][n][0]
    y = RFP_layer[0][n][0]
    z = XFP_layer[0][n][0]

Error: IPyKernel not installed into interpreter Python 3.8.5 64-bit ('base': conda):C:\Users\Peter\anaconda3\python.exe

def f_flat_by_vector(a,b,c,d):
    
for _ in range(len(#temp)):

In [ ]:
# mask_folder = "C:/Users/Peter/Desktop/DATA_TEST/20180210/position740200_CellXOutput/GFP"
# os.chdir(mask_folder)

os.chdir(temp_info)
# os.chdir(microfluidics_results)

tracking = []
count = 0
for root, dirs, files, in os.walk(os.getcwd()):
    for name in files:
        if name.endswith(".mat") and name.startswith("track"):
            tracking.append({'Path': os.path.join(root, name)})
            count = count + 1
            print(count, end="\r")
        else :
            pass

def f_Pos_mask(z):
    start = z.find('_p')+1 #Note: The shift forward is dependant upon how you write out position
    end = start + 7  #Make sure that the 'n' is present in the array to confirm no place 0 has been lost
    return(z[start:end])


def mask_hour(m):
    hourS = m.find("_")+1
    hourE = m.find("_p")
    return(m[hourS:hourE])

def f_maskdate(x):
    dstart = x.find('d0')
    dend = dstart + 5
    expdate = (x[dstart:dend])
    return(expdate)

trackDataIndex = pd.DataFrame(tracking)
trackDataIndex["Date"] = pd.Series(trackDataIndex.iloc[:,0]).apply(f_maskdate)
trackDataIndex["Position_ID"] = pd.Series(trackDataIndex.iloc[:,0]).apply(f_Pos_mask)
trackDataIndex["Hour/Run_t"] = pd.Series(trackDataIndex.iloc[:,0]).apply(mask_hour)
trackDataIndex["Unique_pos_hour"] = trackDataIndex['Date'] + "h" + trackDataIndex["Hour/Run_t"] + trackDataIndex['Position_ID']
del tracking

trackDataIndex

,Path,Date,Position_ID,Hour/Run_t,Unique_pos_hour
0,C:\Users\Peter\Desktop\Subset_Images\COPY - RE...,d0221,C:\User,Images\COPY - RESULTS\d0221r1p500200\GFP_mKO_m...,d0221hImages\COPY - RESULTS\d0221r1p500200\GFP...
1,C:\Users\Peter\Desktop\Subset_Images\COPY - RE...,d0221,C:\User,Images\COPY - RESULTS\d0221r1p500200\GFP_mKO_m...,d0221hImages\COPY - RESULTS\d0221r1p500200\GFP...
2,C:\Users\Peter\Desktop\Subset_Images\COPY - RE...,d0221,C:\User,Images\COPY - RESULTS\d0221r1p500200\GFP_mKO_m...,d0221hImages\COPY - RESULTS\d0221r1p500200\GFP...
3,C:\Users\Peter\Desktop\Subset_Images\COPY - RE...,d0221,C:\User,Images\COPY - RESULTS\d0221r1p500200\GFP_mKO_m...,d0221hImages\COPY - RESULTS\d0221r1p500200\GFP...
4,C:\Users\Peter\Desktop\Subset_Images\COPY - RE...,d0221,C:\User,Images\COPY - RESULTS\d0221r1p500200\GFP_mKO_m...,d0221hImages\COPY - RESULTS\d0221r1p500200\GFP...
...,...,...,...,...,...
64,C:\Users\Peter\Desktop\Subset_Images\COPY - RE...,d0221,C:\User,Images\COPY - RESULTS\d0221r1p500200\GFP_mKO_m...,d0221hImages\COPY - RESULTS\d0221r1p500200\GFP...
65,C:\Users\Peter\Desktop\Subset_Images\COPY - RE...,d0221,C:\User,Images\COPY - RESULTS\d0221r1p500200\GFP_mKO_m...,d0221hImages\COPY - RESULTS\d0221r1p500200\GFP...
66,C:\Users\Peter\Desktop\Subset_Images\COPY - RE...,d0221,C:\User,Images\COPY - RESULTS\d0221r1p500200\GFP_mKO_m...,d0221hImages\COPY - RESULTS\d0221r1p500200\GFP...
67,C:\Users\Peter\Desktop\Subset_Images\COPY - RE...,d0221,C:\User,Images\COPY - RESULTS\d0221r1p500200\GFP_mKO_m...,d0221hImages\COPY - RESULTS\d0221r1p500200\GFP...


In [ ]:
os.chdir(microfluidics_results)
try: imgIndex = pd.read_hdf('MASTER.h5', 'long_imgIndex')
except KeyError: imgIndex = pd.read_hdf('MASTER.h5', 'imgIndex')
else: imgIndex = pd.read_hdf('MASTER.h5', 'long_imgIndex')

temp = imgIndex.drop_duplicates()
temp = temp.reset_index()
temp = temp.set_index(["Unique_pos_hour"])
temp

def barcode_match(q):
    u_frame = temp.loc[[q],["Unique_pos"]]
    u_frame = u_frame.drop_duplicates()
    return(u_frame.iloc[0,0])

trackDataIndex["Unique_pos"] = pd.Series(trackDataIndex.iloc[:,4]).apply(barcode_match)

try: temp
except NameError: pass
else: del temp

del imgIndex

trackDataIndex.set_index(["Unique_pos"], inplace = True)
# trackDataIndex.drop(["Unique_pos_hour"], inplace = True)
trackDataIndex

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'F:/Microfluidics/RESULTS'

In [ ]:
os.chdir(microfluidics_results)

if Experimental_info["data_subset"] == True:
    subset_trackDataIndex = trackDataIndex[subset_trackDataIndex["Unique_pos"].isin(instances)]
    subset_trackDataIndex = subset_trackDataIndex

    trackDataIndex.to_hdf('MASTER.h5', key= 'long_trackIndex', mode='r+')
    subset_trackDataIndex.to_hdf('MASTER.h5', key= 'trackIndex', mode='r+')

    trackDataIndex = subset_trackDataIndex
    del subset_Allmasks
else:
    trackDataIndex.to_hdf('MASTER.h5', key= 'trackIndex', mode='r+')

In [ ]:
tracksHeader = ['track.fileSet.index', 'cell.index', 'cell.frame', 'track.index', 'track.spanned', 'track.child.of.cell', 'track.child.of.track',	'track.absorbs.cell', 'track.absorbs.track']
#This has been hard-coded, but should double check column outputs have not changed between expreriments

In [ ]:
#define function for calling a single position

def track_load(m):
    trackALL = loadmat(m)
    tracking = trackALL['trackerData'][0]['tracks'][0]
    tracking = pd.DataFrame(tracking, columns = tracksHeader)
    return (tracking)

#u_pos = <some variable created in the tracking for loop>
upos_track = track_load(trackDataIndex.iloc[0,0]) # change the iloc to a loc for specifc pos barcode


In [ ]:
u_pos = "d0216r2p030200"
upos_track["cell.index"] = upos_track["cell.index"].astype(str)
upos_track["cell.frame"] = upos_track["cell.frame"].astype(str)
upos_track["Unique_cell"] = u_pos + "c" + upos_track["cell.index"]
upos_track.set_index(["Unique_cell"], inplace= True)
upos_track["cell.index"] = upos_track["cell.index"].astype(int)
upos_track["cell.frame"] = upos_track["cell.frame"].astype(int)

upos_track

,track.fileSet.index,cell.index,cell.frame,track.index,track.spanned,track.child.of.cell,track.child.of.track,track.absorbs.cell,track.absorbs.track
Unique_cell,,,,,,,,,
d0216r2p030200f1c1,1,1,1,1,0,0,0,0,0
d0216r2p030200f1c2,1,2,1,2,0,0,0,0,0
d0216r2p030200f1c3,1,3,1,3,0,0,0,0,0
d0216r2p030200f1c4,1,4,1,4,0,0,0,0,0
d0216r2p030200f1c5,1,5,1,5,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
d0216r2p030200f69c246,69,246,69,260,0,0,0,0,0
d0216r2p030200f69c247,69,247,69,225,0,0,0,0,0
d0216r2p030200f69c248,69,248,69,181,0,0,0,0,0


In [ ]:
final_frame = upos_track[upos_track["cell.frame"] == 69]
final_frame

,track.fileSet.index,cell.index,cell.frame,track.index,track.spanned,track.child.of.cell,track.child.of.track,track.absorbs.cell,track.absorbs.track
Unique_cell,,,,,,,,,
d0216r2p030200f69c1,69,1,69,373,0,0,0,0,0
d0216r2p030200f69c2,69,2,69,360,0,0,0,0,0
d0216r2p030200f69c3,69,3,69,182,0,0,0,0,0
d0216r2p030200f69c4,69,4,69,206,0,0,0,0,0
d0216r2p030200f69c5,69,5,69,408,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
d0216r2p030200f69c246,69,246,69,260,0,0,0,0,0
d0216r2p030200f69c247,69,247,69,225,0,0,0,0,0
d0216r2p030200f69c248,69,248,69,181,0,0,0,0,0


In [ ]:
#Match the cells with their tracking information

In [ ]:
# MaskIndex = Allmasks.index
# # for s in Allmasks.index
# for ind in MaskIndex:
#     Mas_s = simp_mask_load(Allmasks.loc[ind]["Path"])
# #     pix = imgIndex.loc[("d0213r1p110200f0057","GFP"), ["Path"]].iloc[0,0]
#     pix = np.asmatrix(Image.open(pix))
#     pixmas = Mas*pix

In [ ]:
ind = "d0213r1p020300f0001"
mask_r = mask_load(Allmasks.loc[ind]["Path"])

In [ ]:
for cn in range(0:np.max(mask_r))

SyntaxError: invalid syntax (<ipython-input-93-6c6e95ace54f>, line 1)

In [ ]:
Allmasks

,Path,Date,Position_ID,MaskNum,Unique_pos
Unique_frame,,,,,
d0221r1p107030f0001,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0001,d0221r1p107030
d0221r1p107030f0002,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0002,d0221r1p107030
d0221r1p107030f0003,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0003,d0221r1p107030
d0221r1p107030f0004,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0004,d0221r1p107030
d0221r1p107030f0005,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0005,d0221r1p107030
...,...,...,...,...,...
d0222r2p94020f0065,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0065,d0222r2p94020
d0222r2p94020f0066,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0066,d0222r2p94020
d0222r2p94020f0067,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0067,d0222r2p94020


In [ ]:
# os.chdir(microfluidics_results)
# Allmasks = pd.read_hdf('MASTER.h5', 'Allmasks')
tempAllmasks = Allmasks.reset_index()
tempAllmasks

,Unique_frame,Path,Date,Position_ID,MaskNum,Unique_pos
0,d0221r1p107030f0001,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0001,d0221r1p107030
1,d0221r1p107030f0002,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0002,d0221r1p107030
2,d0221r1p107030f0003,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0003,d0221r1p107030
3,d0221r1p107030f0004,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0004,d0221r1p107030
4,d0221r1p107030f0005,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0005,d0221r1p107030
...,...,...,...,...,...,...
2893,d0222r2p94020f0065,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0065,d0222r2p94020
2894,d0222r2p94020f0066,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0066,d0222r2p94020
2895,d0222r2p94020f0067,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0067,d0222r2p94020
2896,d0222r2p94020f0068,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0068,d0222r2p94020


In [ ]:
tempAllmasks

,Unique_frame,Path,Date,Position_ID,MaskNum,Unique_pos
0,d0221r1p107030f0001,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0001,d0221r1p107030
1,d0221r1p107030f0002,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0002,d0221r1p107030
2,d0221r1p107030f0003,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0003,d0221r1p107030
3,d0221r1p107030f0004,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0004,d0221r1p107030
4,d0221r1p107030f0005,F:\Microfluidics\RESULTS\1TemTest\d0221r1p1070...,d0221,1070300,f0005,d0221r1p107030
...,...,...,...,...,...,...
2893,d0222r2p94020f0065,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0065,d0222r2p94020
2894,d0222r2p94020f0066,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0066,d0222r2p94020
2895,d0222r2p94020f0067,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0067,d0222r2p94020
2896,d0222r2p94020f0068,F:\Microfluidics\RESULTS\1TemTest\d0222r2p9402...,d0222,940200,f0068,d0222r2p94020


q = Allmasks.iloc[10, 0]
temp = mask_load(q)

temp

c = 3
a,b = (temp == c).nonzero()
x1 = np.min(a)
x2 = np.max(a)
y1 = np.min(b)
y2 = np.max(b)

print (x1, x2, y1, y2)

In [ ]:
#Insert Cropping.py pipeline
def f_slice(matrix, unq_cell):
    x1 = cell_index.loc(unq_cell, [x1])
    x2 = cell_index.loc(unq_cell, [x2])
    y1 = cell_index.loc(unq_cell, [y1])
    y2 = cell_index.loc(unq_cell, [y2])
    mat = (mat[y1:(y2 + 1), x1:(x2 + 1)])
    return (mat)

# for unq_cell in Cell_index[Cell_index[['Unique_frame'] == unq_frame]]:
#     pix_mat = f_slice(img_pxMAT, unq_cell)
#     cell_mat = f_slice(mask, unq_cell)


Cell_index = pd.DataFrame([])
for q in range(len(Allmasks["Path"])):
    m = tempAllmasks.iloc[q, 1]
    p = tempAllmasks.iloc[q, 8]
    temp = mask_load(m)
    n = np.max(temp)
    for c in range(1, n+1):
        a,b = (temp== c).nonzero()
        x1 = np.min(a)
        x2 = np.max(a)
        y1 = np.min(b)
        y2 = np.max(b)
        u_frame = tempAllmasks.iloc[q, 0]
        t_df = pd.DataFrame([{'u_frame' : u_frame, 'u_pos': p, 'c': c, 'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2}])
        # # t_df = t_df + Allmasks.loc[c]
        
        Cell_index = Cell_index.append(t_df)
    del temp
Cell_index

# Cell_index.to_hdf('MASTER.h5', key ='Cell_index', mode= 'r+')

IndexError: single positional indexer is out-of-bounds

In [ ]:
Cell_index["c"] = Cell_index["c"].astype(str)
Cell_index["u_cell"] = Cell_index["u_pos"] + "c" + Cell_index["c"]
Cell_index["u_cell_frame"] = Cell_index["u_frame"] + "c" + Cell_index["c"]
# Cell_index.drop(["u_frame", "c"], axis = 1, inplace = True)
Cell_index.group_by(["u_cell", "u_cell_frame"], inplace= True)

In [ ]:
Cell_index.to_hdf("MASTER.h5", 'Cell_index')

In [ ]:
os.chdir(microfluidics_results)
Cell_index = pd.read_hdf("MASTER.h5", 'Cell_index')
Cell_index

,u_frame,u_pos,c,x1,x2,y1,y2
0,d0213r1p020300f0001,d0213r1p020300,1,1358,1376,25,44
0,d0213r1p020300f0001,d0213r1p020300,2,970,1000,23,52
0,d0213r1p020300f0001,d0213r1p020300,3,724,747,30,55
0,d0213r1p020300f0001,d0213r1p020300,4,1672,1687,72,86
0,d0213r1p020300f0001,d0213r1p020300,5,598,619,72,91
...,...,...,...,...,...,...,...
0,d0213r1p020300f0002,d0213r1p020300,406,1303,1325,1374,1397
0,d0213r1p020300f0002,d0213r1p020300,407,1864,1886,1374,1396
0,d0213r1p020300f0002,d0213r1p020300,408,209,230,1380,1400
0,d0213r1p020300f0002,d0213r1p020300,409,1768,1788,1388,1411


In [ ]:
def max2_min2(u):
    np.max
# Cell_index["x_min"] = np.max(pd.Series(Cell_index.loc["u_cell","x1"])
Cell_index["y_min"] = np.max(pd.Series(Cell_index.loc["u_cell","y1"])
Cell_index["x_max"] = np.max(pd.Series(Cell_index.loc["u_cell", "x2"])
Cell_index["y_max"] = np.max(pd.Series(Cell_index.loc["u_cell", "y2"])

SyntaxError: invalid syntax (<ipython-input-11-e236271a20b3>, line 3)

In [ ]:
try: imgIndex
except NameError: pass
else: del imgIndex

try: Allmasks
except NameError: pass
else: del Allmasks

try: tempAllmasks
except NameError: pass
else: del tempAllmasks


os.chdir(microfluidics_results)
imgIndex_TP = pd.read_hdf('MASTER.h5', 'imgIndex')
imgIndex_TP.reset_index()
segIndex_TP = pd.read_hdf('MASTER.h5', 'Allmasks')
segIndex_TP.reset_index()

threshold = 99 #This is something that must be tuned

imgIndex_TP = pd.read_hdf('MASTER.h5', 'imgIndex')
segIndex_TP = pd.read_hdf('MASTER.h5', 'Allmasks')

pixvalTP_DF = pd.DataFrame([])
count = 0
start_index = 0
end_index = len(imgIndex_TP)


# FOR LOOP START #
############################################################################################
print('function started')
@njit
for i in np.arange(start_index, end_index, 1):

    os.chdir(image_path)
    img_pixel = np.asmatrix(Image.open(imgIndex_TP.iloc[i,0]))
    
    if np.amax(img_temp) == 0:
        continue

    strainID = imgIndex_TP.iloc[i,3]
    imageID = imgIndex_TP.iloc[i,2]
    TimePoint = imgIndex_TP.iloc[i,1]
    seg_path = segIndex_TP[segIndex_TP['TimePoint'] == TimePoint]
    seg_path = seg_path[seg_path['ImageID'] == imageID]

    if len(seg_path) == 0:
        continue
    
    FilteredObjects_Path = seg_path.iloc[0,0]
    
    os.chdir(FilteredObjects_Path)
    
    FilteredObjects = []
    for root, dirs, files in os.walk(os.getcwd()):
        for j in files:
            if j.endswith('tiff'):
                FilteredObjects.append({'ObjectFile': j})
            
    FilteredObjects = pd.DataFrame(FilteredObjects)
    
    if len(FilteredObjects) == 0:
        continue

    objMeas_DF = pd.DataFrame([])

    for k in np.arange(0, len(FilteredObjects), 1):

        object_measurements = []
        
        object_pxMAT = np.asmatrix(Image.open(FilteredObjects.iloc[k,0]))
        object_pxMAT.setflags(write=1)
        object_pxMAT[object_pxMAT == 255] = 1

        rawGFP_object = np.multiply(object_pxMAT, img_pxMAT)
        rawGFP_object = np.array(rawGFP_object, dtype = np.uint8)

        rawGFP_object = rawGFP_object[rawGFP_object > 0] 

        factor = np.median(rawGFP_object)
        normGFP_object = rawGFP_object/factor
        
        normGFP_object = np.percentile(normGFP_object, threshold)
        #normGFP_object = np.percentile(normGFP_object, threshold) - np.percentile(normGFP_object, (100-threshold))
        
        x95th = np.median(normGFP_object)
        CellSize = len(rawGFP_object)
                    
        object_measurements = {'StrainID' : strainID,
                               'TimePoint' : TimePoint,
                               'ObjectNumber' : FilteredObjects.iloc[k,0],
                               'x95thPercentile' : [x95th],
                               'CellSize' : [CellSize],
                               'CellMedian' : [factor]}

        object_measurements = pd.DataFrame(object_measurements)
        object_measurements.columns = ['StrainID', 'TimePoint', 
                                       'ObjectNumber',
                                       '95thPercentile',
                                       'CellSize',
                                       'CellMedian']
        
        objMeas_DF = pd.concat([objMeas_DF, object_measurements])

    pixvalTP_DF = pd.concat([pixvalTP_DF, objMeas_DF])
    count = count + 1
    print("Progress {:2.1%}".format(count / (end_index-start_index)), end="\r")
print('function ended')